In [1]:
from merlin.io import Dataset
from nvtabular import ops
import nvtabular as nvt
from merlin.schema.tags import Tags

from merlin.schema import Schema, ColumnSchema, Tags

# Initialize an empty schema
schema = Schema()

article_id_fixed_col = ColumnSchema(
    name="article_id_fixed",
    dtype="int32",
    tags=[Tags.LIST, Tags.CATEGORICAL, Tags.ITEM_ID, Tags.ITEM],
    is_list=True,
    is_ragged=True,
).with_properties({'domain': {'min': 0, 'max': 125540}, 'value_count': {'min': 1, 'max': 500}})

read_time_fixed_col = ColumnSchema(
    name="read_time_fixed",
    dtype="float32",
    tags=[Tags.LIST, Tags.CONTINUOUS],
    is_list=True,
    is_ragged=True
)

inview_col = ColumnSchema(
    name="article_ids_inview",
    dtype="int32",
    tags=[Tags.CATEGORICAL]
).with_properties({'domain': {'min': 0, 'max': 125540}, 'value_count': {'min': 1, 'max': 500}})

label_col = ColumnSchema(
    name="label",
    dtype="int8",
    tags=[Tags.TARGET, Tags.BINARY_CLASSIFICATION]
)

# Add columns to the schema
columns = [article_id_fixed_col, read_time_fixed_col]

for col in columns:
    schema[col.name] = col

/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [2]:
schema

,name,tags,dtype,is_list,is_ragged,properties.value_count.min,properties.value_count.max,properties.domain.min,properties.domain.max
0,article_id_fixed,"(Tags.ITEM, Tags.LIST, Tags.ID, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",True,True,1,500.0,0.0,125540.0
1,read_time_fixed,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,0,NaN,NaN,NaN


In [3]:
train_df = Dataset("/home/scur1565/News-Recommender-Recsys24/pipeline_end_to_end_functional/data/normal/train_data_small.parquet", schema=schema, engine="parquet")

In [4]:
train_df.head(2)

,article_ids_clicked,event_time_ts,articles_history_len,scroll_percentage_fixed,article_id_fixed,read_time_fixed,device_type,article_ids_inview,label
0,[123341],1684914473000000000,19,"[100.0, 89.0, 27.0, 33.0, 100.0, 75.0, 39.0, 2...","[118684, 119080, 119059, 119011, 119431, 11944...","[60.0, 11.0, 1.0, 15.0, 37.0, 15.0, 4.0, 8.0, ...",2,"[123336, 123346, 123345, 123341, 123356, 123353]",0
1,[123336],1684913605000000000,482,"[21.0, 100.0, 34.0, 85.0, 92.0, 75.0, 52.0, 66...","[117711, 117836, 117827, 117819, 117822, 11767...","[7.0, 24.0, 28.0, 65.0, 16.0, 41.0, 59.0, 24.0...",2,"[123350, 123353, 123357, 123345, 123341, 12335...",0


In [5]:
from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt

max_sequence_length, d_model = 20, 64
emb_dims = {"article_id_fixed": 768}
infer_embedding_sizes = True

inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=max_sequence_length,
        continuous_projection=d_model,
        aggregation="concat",
        d_output=100,
        masking='causal',
        infer_embedding_sizes=infer_embedding_sizes,
        embedding_dims=emb_dims
)

/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch
import polars as pl

pretrained_embeds_df = pl.read_parquet("../data/eb_contrastive_vector/contrastive_vector.parquet")

In [7]:
article_to_index = {
    art_id: num for num, art_id in enumerate(pretrained_embeds_df['article_id'].to_list())
}

In [8]:
pretrained_embeds_df.head(3)

article_id,contrastive_vector
i32,list[f32]
3000022,"[-0.012159, 0.057097, … 0.013574]"
3000063,"[0.034482, 0.033533, … -0.023085]"
3000613,"[-0.014638, 0.030934, … -0.006551]"


In [9]:
import numpy as np

pretrained_embeds = pretrained_embeds_df['contrastive_vector'].to_list()
pretrained_embeds = np.vstack([np.array(vec) for vec in pretrained_embeds])

In [10]:
pretrained_embeds = torch.from_numpy(pretrained_embeds)

In [11]:
schema

,name,tags,dtype,is_list,is_ragged,properties.value_count.min,properties.value_count.max,properties.domain.min,properties.domain.max
0,article_id_fixed,"(Tags.ITEM, Tags.LIST, Tags.ID, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",True,True,1,500.0,0.0,125540.0
1,read_time_fixed,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,0,NaN,NaN,NaN


In [38]:
tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=max_sequence_length,
        continuous_projection=d_model,
        aggregation="concat",
        d_output=100,
        masking='causal',
        infer_embedding_sizes=True,
        embedding_dims={"article_id_fixed": 768}
)

TabularSequenceFeatures(
  (_aggregation): ConcatFeatures()
  (to_merge): ModuleDict(
    (continuous_module): SequentialBlock(
      (0): ContinuousFeatures(
        (filter_features): FilterFeatures()
        (_aggregation): ConcatFeatures()
      )
      (1): SequentialBlock(
        (0): DenseBlock(
          (0): Linear(in_features=1, out_features=64, bias=True)
          (1): ReLU(inplace=True)
        )
      )
      (2): AsTabular()
    )
    (categorical_module): SequenceEmbeddingFeatures(
      (filter_features): FilterFeatures()
      (embedding_tables): ModuleDict(
        (article_id_fixed): Embedding(125541, 768, padding_idx=0)
      )
    )
    (pretrained_embedding_module): PretrainedEmbeddingFeatures(
      (filter_features): FilterFeatures()
    )
  )
  (projection_module): SequentialBlock(
    (0): DenseBlock(
      (0): Linear(in_features=832, out_features=100, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (_masking): CausalLanguageModeling()
)

In [39]:
with torch.no_grad():
    inputs.categorical_module.embedding_tables["article_id_fixed"].weight.copy_(
        pretrained_embeds
    )

inputs.categorical_module.embedding_tables["article_id_fixed"].requires_grad = False
inputs.categorical_module.embedding_tables[
    "article_id_fixed"
].weight.requires_grad = False

In [40]:
# from transformers4rec.torch.ranking_metric import Precision,Recall,Accuracy

# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=max_sequence_length
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([64]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the evaluation top-N metrics and the cut-offs
metrics = [NDCGAt(top_ks=[20, 40], labels_onehot=True),  
           RecallAt(top_ks=[20, 40], labels_onehot=True)]

# Define a head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(
        weight_tying=True,
        metrics=metrics
    )
    ,inputs=inputs
)

# Get the end-to-end Model class 
model = tr.Model(head)

Projecting inputs of NextItemPredictionTask to'768' As weight tying requires the input dimension '64' to be equal to the item-id embedding dimension '768'


### Train Model

In [41]:
per_device_train_batch_size: int = 128
per_device_eval_batch_size: int = 32

In [42]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    dataloader_drop_last = True,
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = per_device_train_batch_size, 
                                    per_device_eval_batch_size = per_device_eval_batch_size,
                                    output_dir = "./tmp", 
                                    learning_rate=0.0005,
                                    lr_scheduler_type='cosine', 
                                    learning_rate_num_cosine_cycles_by_epoch=1.5,
                                    num_train_epochs=10,
                                    max_sequence_length=20, 
                                    report_to = [],
                                    logging_steps=200,
                                    no_cuda=False)

In [43]:
# Instantiate the T4Rec Trainer, which manages training and evaluation for the PyTorch API
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [44]:
train_paths = f"/home/scur1565/News-Recommender-Recsys24/pipeline_end_to_end_functional/data/normal/train_data_small.parquet"
eval_paths = f"/home/scur1565/News-Recommender-Recsys24/pipeline_end_to_end_functional/data/normal/validation_data_small.parquet"
print(train_paths)

trainer.train_dataset_or_path = train_paths
trainer.reset_lr_scheduler()
trainer.train()
trainer.state.global_step += 1
print('Finished training')

# trainer.eval_dataset_or_path = eval_paths
# train_metrics = trainer.evaluate(metric_key_prefix='eval')

# for key in sorted(train_metrics.keys()):
#     print(" %s = %s" % (key, str(train_metrics[key]))) 
    
# from transformers4rec.torch.utils.examples_utils import wipe_memory
# wipe_memory()

/home/scur1565/News-Recommender-Recsys24/pipeline_end_to_end_functional/data/normal/train_data_small.parquet


/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,10.924600
400,10.246600
600,10.018600
800,9.986700
1000,9.868300
1200,9.569300
1400,9.265100
1600,9.024300
1800,8.879200
2000,8.846000


KeyboardInterrupt: 

In [19]:
import pandas as pd

In [22]:
validation_df = pd.read_parquet("/home/scur1565/News-Recommender-Recsys24/pipeline_end_to_end_functional/data/normal/validation_data_small.parquet", engine="pyarrow")
all_list_ids_inview: List[List[int]] = validation_df["article_ids_inview"].to_list()
all_list_ids_clicked: List[List[int]] = validation_df["article_ids_clicked"].to_list()

dlv = trainer.get_eval_dataloader()


In [26]:
validation_df['label'] = validation_df.apply(lambda x: [1 if inview_id in x.article_ids_clicked else 0 for inview_id in x.article_ids_inview], axis=1)
validation_df.head(2)

,article_ids_clicked,event_time_ts,articles_history_len,scroll_percentage_fixed,article_id_fixed,read_time_fixed,device_type,article_ids_inview,label
0,[124070],1685247684000000000,130,"[29.0, -1.0, 79.0, 17.0, 68.0, 37.0, 92.0, 15....","[119437, 119007, 119510, 119502, 119493, 11968...","[6.0, 0.0, 13.0, 16.0, 13.0, 7.0, 50.0, 12.0, ...",2,"[124015, 124062, 124069, 124070, 124072]","[0, 0, 0, 1, 0]"
1,[124045],1685248308000000000,130,"[29.0, -1.0, 79.0, 17.0, 68.0, 37.0, 92.0, 15....","[119437, 119007, 119510, 119502, 119493, 11968...","[6.0, 0.0, 13.0, 16.0, 13.0, 7.0, 50.0, 12.0, ...",2,"[123883, 124015, 123865, 113026, 123872, 12404...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [27]:
print(f"Ensure that Validation Dataloader is deterministic: {not dlv.shuffle}")

Ensure that Validation Dataloader is deterministic: True


In [32]:
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from typing import List, Tuple
from tqdm import tqdm

In [31]:
def get_inview_articles_score(
    ids_inview: List[int], 
    ids_clicked: List[int], 
    prob_list: List[float]
) -> List[float]:

    inview_scores: List[float] = []
    
    for inview_id in ids_inview:
       inview_scores.append(prob_list[inview_id]) 

    return inview_scores

In [45]:
all_inview_scores: List[List[float]] = []
index: int = 0
# Sample one from data loader
for sp_batch in tqdm(dlv):
    # pass through body - encoding
    body_out = body(sp_batch[0])
    # pass encoding through the prediction head
    head_out = head(body_out)
    # extract probabilities
    list_of_probs = head_out['next-item'][0].cpu().tolist()
    # get inview prob scores
    for sample in head_out:
        ids_clicked = all_list_ids_clicked[index]
        ids_inview = all_list_ids_inview[index]
        inview_scores = get_inview_articles_score(ids_inview, ids_clicked, list_of_probs)

        all_inview_scores.append(inview_scores)

        index += 1

100%|██████████| 7645/7645 [01:12<00:00, 106.13it/s]


In [46]:
metrics = MetricEvaluator(
    labels=validation_df["label"].to_list(),
    predictions=all_inview_scores,
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

In [ ]:
from ebrec.utils._python import write_submission_file, rank_predictions_by_score

In [ ]:
validation_df["scores"] = all_inview_scores

In [ ]:
validation_df = validation_df.with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
validation_df.head(2)